# COBOL Analysis Engine v2.0 - SDK Complete Example

Este notebook demonstra todas as funcionalidades do SDK do COBOL Analysis Engine v2.0.

## Índice
1. [Configuração Inicial](#configuracao)
2. [Análise Básica](#analise-basica)
3. [Análise com Copybooks](#analise-copybooks)
4. [Análise em Lote](#analise-lote)
5. [Comparação de Modos](#comparacao-modos)
6. [Visualizações](#visualizacoes)
7. [Exportação](#exportacao)
8. [Interface Interativa](#interface-interativa)
9. [Configurações Avançadas](#configuracoes-avancadas)

## 1. Configuração Inicial {#configuracao}

Primeiro, vamos importar o SDK e configurar o ambiente.

In [ ]:
# Importações necessárias
import sys
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, Markdown, HTML

# Adicionar o diretório do SDK ao path
sys.path.append('../src/api')
sys.path.append('../src')

# Importar o SDK
from cobol_analyzer_sdk import COBOLAnalyzer, JupyterCOBOLAnalyzer, quick_analyze

print("✅ SDK importado com sucesso!")
print(f"📁 Diretório atual: {os.getcwd()}")

In [ ]:
# Inicializar o analisador
analyzer = COBOLAnalyzer()

# Verificar modos disponíveis
available_modes = analyzer.get_available_modes()
print(f"🔧 Modos disponíveis: {available_modes}")

# Configurar estilo dos gráficos
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("✅ Analisador inicializado!")

## 2. Análise Básica {#analise-basica}

Vamos começar com uma análise básica de um programa COBOL.

In [ ]:
# Análise de arquivo individual
program_file = "LHAN0542_TESTE.cbl"

print(f"🔍 Analisando programa: {program_file}")
print("⏳ Aguarde...")

# Executar análise no modo enhanced
result = analyzer.analyze_file(
    file_path=program_file,
    mode="enhanced",
    output_dir="notebook_output/"
)

print(f"✅ Status: {result['status']}")
if result['status'] == 'success':
    print(f"📄 Relatório salvo em: {result.get('report_path', 'N/A')}")
    print(f"⏱️ Tempo de execução: {result.get('execution_time', 0):.2f}s")
else:
    print(f"❌ Erro: {result.get('error', 'Erro desconhecido')}")

In [ ]:
# Exibir resumo da análise
if result['status'] == 'success':
    summary = analyzer.get_analysis_summary(result)
    
    print("📊 Resumo da Análise:")
    print("=" * 50)
    
    # Criar DataFrame para melhor visualização
    summary_df = pd.DataFrame(list(summary.items()), columns=['Métrica', 'Valor'])
    display(summary_df)
    
    # Exibir primeiras linhas da documentação
    doc_preview = result['documentation'][:1000] + "..." if len(result['documentation']) > 1000 else result['documentation']
    print("\n📝 Preview da Documentação:")
    print("=" * 50)
    display(Markdown(doc_preview))

## 3. Análise com Copybooks {#analise-copybooks}

Agora vamos analisar um programa incluindo copybooks para uma análise mais completa.

In [ ]:
# Análise com copybooks
program_file = "LHAN0542_TESTE.cbl"
books_file = "BOOKS.txt"

print(f"🔍 Analisando programa: {program_file}")
print(f"📚 Com copybooks: {books_file}")
print("⏳ Aguarde...")

result_with_books = analyzer.analyze_file(
    file_path=program_file,
    mode="enhanced",
    output_dir="notebook_output_with_books/",
    copybooks_path=books_file
)

print(f"✅ Status: {result_with_books['status']}")
if result_with_books['status'] == 'success':
    print(f"📄 Relatório salvo em: {result_with_books.get('report_path', 'N/A')}")
    print(f"⏱️ Tempo de execução: {result_with_books.get('execution_time', 0):.2f}s")
    
    # Comparar com análise anterior
    summary_with_books = analyzer.get_analysis_summary(result_with_books)
    summary_without_books = analyzer.get_analysis_summary(result)
    
    comparison_df = pd.DataFrame({
        'Sem Copybooks': summary_without_books,
        'Com Copybooks': summary_with_books
    })
    
    print("\n📊 Comparação:")
    display(comparison_df)

## 4. Análise em Lote {#analise-lote}

Vamos processar múltiplos programas de uma vez usando o arquivo fontes.txt.

In [ ]:
# Configurar callback de progresso
from tqdm.notebook import tqdm

progress_bar = None

def progress_callback(current, total, program_name):
    global progress_bar
    if progress_bar is None:
        progress_bar = tqdm(total=total, desc="Processando")
    progress_bar.update(1)
    progress_bar.set_description(f"Processando {program_name}")

def result_callback(program_name, result):
    status_icon = "✅" if result['status'] == 'success' else "❌"
    print(f"{status_icon} {program_name}: {result['status']}")

print("🚀 Iniciando processamento em lote...")

In [ ]:
# Executar análise em lote
batch_results = analyzer.analyze_batch(
    fontes_path="fontes.txt",
    books_path="BOOKS.txt",
    mode="enhanced",
    output_dir="notebook_batch_output/",
    progress_callback=progress_callback,
    result_callback=result_callback,
    max_programs=5  # Limitar para demonstração
)

if progress_bar:
    progress_bar.close()

print(f"\n🎉 Processamento concluído!")
print(f"📊 Total de programas: {len(batch_results)}")
print(f"✅ Sucessos: {len([r for r in batch_results if r['status'] == 'success'])}")
print(f"❌ Falhas: {len([r for r in batch_results if r['status'] == 'error'])}")

In [ ]:
# Criar DataFrame dos resultados
batch_df = pd.DataFrame([
    analyzer.get_analysis_summary(result) for result in batch_results
])

print("📋 Resumo dos Resultados:")
display(batch_df)

# Estatísticas básicas
print("\n📈 Estatísticas:")
print(f"⏱️ Tempo médio de execução: {batch_df['execution_time'].mean():.2f}s")
print(f"📄 Média de linhas de documentação: {batch_df['documentation_lines'].mean():.0f}")
print(f"💾 Tamanho médio dos arquivos: {batch_df['file_size_kb'].mean():.1f} KB")

## 5. Comparação de Modos {#comparacao-modos}

Vamos comparar os diferentes modos de análise disponíveis.

In [ ]:
# Testar todos os modos disponíveis
test_program = "LHAN0542_TESTE.cbl"
mode_results = {}

for mode in analyzer.get_available_modes():
    print(f"🔄 Testando modo: {mode}")
    
    result = analyzer.analyze_file(
        file_path=test_program,
        mode=mode,
        output_dir=f"notebook_mode_comparison_{mode}/"
    )
    
    mode_results[mode] = result
    
    if result['status'] == 'success':
        print(f"  ✅ Sucesso - {result.get('execution_time', 0):.2f}s")
    else:
        print(f"  ❌ Erro: {result.get('error', 'Desconhecido')}")

print("\n🏁 Comparação de modos concluída!")

In [ ]:
# Criar comparação detalhada
comparison_data = []

for mode, result in mode_results.items():
    if result['status'] == 'success':
        summary = analyzer.get_analysis_summary(result)
        summary['mode'] = mode
        comparison_data.append(summary)

comparison_df = pd.DataFrame(comparison_data)

print("📊 Comparação Detalhada dos Modos:")
display(comparison_df[['mode', 'execution_time', 'documentation_lines', 'file_size_kb']])

# Gráfico comparativo
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Tempo de execução
comparison_df.plot(x='mode', y='execution_time', kind='bar', ax=axes[0], 
                  title='Tempo de Execução (s)', legend=False, color='skyblue')
axes[0].set_ylabel('Segundos')

# Linhas de documentação
comparison_df.plot(x='mode', y='documentation_lines', kind='bar', ax=axes[1], 
                  title='Linhas de Documentação', legend=False, color='lightgreen')
axes[1].set_ylabel('Linhas')

# Tamanho do arquivo
comparison_df.plot(x='mode', y='file_size_kb', kind='bar', ax=axes[2], 
                  title='Tamanho do Arquivo (KB)', legend=False, color='salmon')
axes[2].set_ylabel('KB')

plt.tight_layout()
plt.show()

## 6. Visualizações {#visualizacoes}

Vamos criar visualizações avançadas dos resultados da análise em lote.

In [ ]:
# Visualizações dos resultados do lote
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# 1. Status das análises
status_counts = batch_df['status'].value_counts()
axes[0,0].pie(status_counts.values, labels=status_counts.index, autopct='%1.1f%%', 
              colors=['lightgreen', 'lightcoral'])
axes[0,0].set_title('Status das Análises')

# 2. Tempo de execução por programa
successful_results = batch_df[batch_df['status'] == 'success']
if not successful_results.empty:
    successful_results.plot(x='program_name', y='execution_time', kind='bar', 
                           ax=axes[0,1], title='Tempo de Execução por Programa', 
                           legend=False, color='steelblue')
    axes[0,1].set_ylabel('Segundos')
    axes[0,1].tick_params(axis='x', rotation=45)

# 3. Distribuição do tamanho dos arquivos
if not successful_results.empty:
    axes[1,0].hist(successful_results['file_size_kb'], bins=10, 
                   color='lightblue', alpha=0.7, edgecolor='black')
    axes[1,0].set_title('Distribuição do Tamanho dos Arquivos')
    axes[1,0].set_xlabel('Tamanho (KB)')
    axes[1,0].set_ylabel('Frequência')

# 4. Correlação entre tamanho e tempo
if not successful_results.empty and len(successful_results) > 1:
    axes[1,1].scatter(successful_results['file_size_kb'], 
                     successful_results['execution_time'], 
                     alpha=0.7, color='orange')
    axes[1,1].set_title('Tamanho vs Tempo de Execução')
    axes[1,1].set_xlabel('Tamanho (KB)')
    axes[1,1].set_ylabel('Tempo (s)')
    
    # Linha de tendência
    z = np.polyfit(successful_results['file_size_kb'], 
                   successful_results['execution_time'], 1)
    p = np.poly1d(z)
    axes[1,1].plot(successful_results['file_size_kb'], 
                   p(successful_results['file_size_kb']), "r--", alpha=0.8)

plt.tight_layout()
plt.show()

In [ ]:
# Heatmap de métricas (se temos dados suficientes)
if not successful_results.empty and len(successful_results) > 2:
    # Selecionar colunas numéricas para o heatmap
    numeric_cols = ['execution_time', 'documentation_lines', 'file_size_kb']
    correlation_data = successful_results[numeric_cols].corr()
    
    plt.figure(figsize=(8, 6))
    sns.heatmap(correlation_data, annot=True, cmap='coolwarm', center=0,
                square=True, linewidths=0.5)
    plt.title('Correlação entre Métricas de Análise')
    plt.tight_layout()
    plt.show()
else:
    print("📊 Dados insuficientes para heatmap de correlação")

## 7. Exportação {#exportacao}

Vamos demonstrar as diferentes opções de exportação disponíveis.

In [ ]:
# Exportar resultado para diferentes formatos
if result['status'] == 'success':
    print("📤 Exportando resultado para diferentes formatos...")
    
    # JSON
    json_path = analyzer.export_to_format(result, format="json", 
                                          output_path="notebook_export/analysis.json")
    print(f"📄 JSON exportado: {json_path}")
    
    # HTML
    html_path = analyzer.export_to_format(result, format="html", 
                                          output_path="notebook_export/analysis.html")
    print(f"🌐 HTML exportado: {html_path}")
    
    try:
        # Excel (se openpyxl estiver disponível)
        excel_path = analyzer.export_to_format(result, format="excel", 
                                               output_path="notebook_export/analysis.xlsx")
        print(f"📊 Excel exportado: {excel_path}")
    except ImportError:
        print("⚠️ Excel export requer openpyxl: pip install openpyxl")
    
    try:
        # PDF (se weasyprint estiver disponível)
        pdf_path = analyzer.export_to_format(result, format="pdf", 
                                             output_path="notebook_export/analysis.pdf")
        print(f"📑 PDF exportado: {pdf_path}")
    except ImportError:
        print("⚠️ PDF export requer weasyprint: pip install weasyprint")
    
    print("\n✅ Exportação concluída!")
else:
    print("❌ Não é possível exportar - análise falhou")

In [ ]:
# Criar relatório comparativo
if len(batch_results) > 1:
    print("📊 Criando relatório comparativo...")
    
    comparison_report_path = analyzer.create_comparison_report(
        batch_results, 
        output_path="notebook_export/comparison_report.html"
    )
    
    print(f"📈 Relatório comparativo criado: {comparison_report_path}")
    
    # Exibir preview do relatório
    with open(comparison_report_path, 'r', encoding='utf-8') as f:
        html_content = f.read()
    
    # Mostrar apenas o início do HTML
    preview = html_content[:2000] + "..." if len(html_content) > 2000 else html_content
    print("\n📝 Preview do HTML:")
    print(preview[:500] + "...")
else:
    print("📊 Dados insuficientes para relatório comparativo")

## 8. Interface Interativa {#interface-interativa}

Vamos criar uma interface interativa usando widgets do Jupyter.

In [ ]:
# Inicializar analisador especializado para Jupyter
jupyter_analyzer = JupyterCOBOLAnalyzer()

print("🎛️ Analisador Jupyter inicializado!")
print("📝 Execute a próxima célula para ver a interface interativa.")

In [ ]:
# Interface interativa (requer ipywidgets)
try:
    jupyter_analyzer.interactive_analyze()
except Exception as e:
    print(f"⚠️ Interface interativa não disponível: {e}")
    print("💡 Para usar widgets interativos, instale: pip install ipywidgets")
    print("💡 E habilite no Jupyter: jupyter nbextension enable --py widgetsnbextension")

In [ ]:
# Análise de código inline (sem arquivo)
cobol_code_example = """
IDENTIFICATION DIVISION.
PROGRAM-ID. EXEMPLO-NOTEBOOK.

ENVIRONMENT DIVISION.
CONFIGURATION SECTION.

DATA DIVISION.
WORKING-STORAGE SECTION.
01 WS-CONTADOR PIC 9(3) VALUE ZEROS.
01 WS-NOME     PIC X(30) VALUE SPACES.

PROCEDURE DIVISION.
0000-PRINCIPAL SECTION.
    DISPLAY 'EXEMPLO DE ANÁLISE EM NOTEBOOK'.
    MOVE 100 TO WS-CONTADOR.
    MOVE 'TESTE COBOL' TO WS-NOME.
    DISPLAY 'CONTADOR: ' WS-CONTADOR.
    DISPLAY 'NOME: ' WS-NOME.
    STOP RUN.
"""

print("🔍 Analisando código COBOL inline...")

inline_result = analyzer.analyze_code(
    cobol_code=cobol_code_example,
    program_name="EXEMPLO-NOTEBOOK",
    mode="enhanced"
)

if inline_result['status'] == 'success':
    print("✅ Análise inline concluída!")
    
    # Exibir resumo
    inline_summary = analyzer.get_analysis_summary(inline_result)
    summary_df = pd.DataFrame(list(inline_summary.items()), columns=['Métrica', 'Valor'])
    display(summary_df)
    
    # Exibir documentação
    print("\n📝 Documentação Gerada:")
    display(Markdown(inline_result['documentation'][:1500] + "..."))
else:
    print(f"❌ Erro na análise inline: {inline_result.get('error', 'Desconhecido')}")

## 9. Configurações Avançadas {#configuracoes-avancadas}

Vamos explorar configurações avançadas e personalização do SDK.

In [ ]:
# Configuração customizada
custom_config = {
    'system': {
        'version': '2.0',
        'log_level': 'DEBUG',
        'timeout_seconds': 600
    },
    'analysis': {
        'default_mode': 'enhanced',
        'max_file_size_mb': 100
    },
    'output': {
        'format': 'markdown',
        'include_prompts': True,
        'include_raw_responses': True
    },
    'cache': {
        'enabled': True,
        'ttl_hours': 48
    }
}

# Criar analisador com configuração customizada
custom_analyzer = COBOLAnalyzer(custom_config=custom_config)

print("⚙️ Analisador customizado criado!")
print(f"🔧 Modos disponíveis: {custom_analyzer.get_available_modes()}")

In [ ]:
# Configurar provedor de IA (exemplo)
# Nota: Requer chave de API válida

# custom_analyzer.configure_provider('openai', {
#     'api_key': 'sua_chave_aqui',
#     'model': 'gpt-4-turbo',
#     'temperature': 0.1,
#     'max_tokens': 4000
# })

print("🤖 Configuração de provedor de IA (exemplo comentado)")
print("💡 Descomente e configure com sua chave de API para usar")

In [ ]:
# Validação de sintaxe COBOL
test_codes = [
    # Código válido
    """
    IDENTIFICATION DIVISION.
    PROGRAM-ID. TESTE-VALIDO.
    """,
    
    # Código inválido
    """
    PROGRAM-ID. SEM-IDENTIFICATION.
    """
]

print("🔍 Testando validação de sintaxe COBOL:")
print("=" * 50)

for i, code in enumerate(test_codes, 1):
    validation = analyzer.validate_cobol_syntax(code)
    
    print(f"\n📝 Teste {i}:")
    print(f"✅ Válido: {validation['is_valid']}")
    
    if validation['errors']:
        print("❌ Erros:")
        for error in validation['errors']:
            print(f"  - {error}")
    
    if validation['warnings']:
        print("⚠️ Avisos:")
        for warning in validation['warnings']:
            print(f"  - {warning}")

In [ ]:
# Demonstração de funções de conveniência
print("🚀 Testando funções de conveniência:")
print("=" * 50)

# Análise rápida
try:
    quick_result = quick_analyze("LHAN0542_TESTE.cbl", mode="traditional")
    print(f"⚡ Análise rápida: {quick_result['status']}")
    print(f"⏱️ Tempo: {quick_result.get('execution_time', 0):.2f}s")
except Exception as e:
    print(f"❌ Erro na análise rápida: {e}")

print("\n✅ Demonstração de configurações avançadas concluída!")

## Resumo e Próximos Passos

Este notebook demonstrou todas as principais funcionalidades do COBOL Analysis Engine v2.0 SDK:

### ✅ Funcionalidades Demonstradas

1. **Análise Básica**: Análise individual de programas COBOL
2. **Análise com Copybooks**: Integração de copybooks para análise completa
3. **Análise em Lote**: Processamento de múltiplos programas
4. **Comparação de Modos**: Comparação entre traditional, multi_ai e enhanced
5. **Visualizações**: Gráficos e estatísticas dos resultados
6. **Exportação**: Múltiplos formatos (JSON, HTML, Excel, PDF)
7. **Interface Interativa**: Widgets para análise interativa
8. **Configurações Avançadas**: Personalização e validação

### 🚀 Próximos Passos

1. **Configurar Provedores de IA**: Adicione suas chaves de API para análises completas
2. **Personalizar Prompts**: Adapte os prompts para seu contexto específico
3. **Integrar em Workflows**: Use o SDK em seus pipelines de análise
4. **Explorar Configurações**: Teste diferentes configurações para otimizar performance
5. **Automatizar Processos**: Crie scripts automatizados para análises regulares

### 📚 Recursos Adicionais

- **Manual Completo**: `docs/MANUAL_COMPLETO_CONFIGURACAO_USO.md`
- **Documentação da API**: `src/api/cobol_analyzer_sdk.py`
- **Exemplos**: Diretório `examples/`
- **Configurações**: Arquivo `config/config.yaml`

---

**COBOL Analysis Engine v2.0** - Análise Inteligente de Código COBOL  
*Desenvolvido com ❤️ para modernização de sistemas legados*